In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file, TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [3]:
# cluster
# cluster_netgen
# compression
# expansion
# explosion
# grid
# implosion
# linearprojection
# rotation
# uniform_portgen

In [4]:
N = 30
instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json")

number_of_instances = N // 5
train_instances = InstanceList()
test_instances = InstanceList()
for i in range(10):
    test_instances.append(instances[i * 50])
for i in range(1, 6):
    train_instances.extend(instances[i * 50 + 1: i * 50 + number_of_instances + 1])

In [5]:
for instance in test_instances:
    instance.log()

[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/cluster_netgen/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/compression/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/expansion/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/grid/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/implosion/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/linearprojection/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/00.tsp)
[2024-12-30 22:43:54] DEBUG     TSP_Instance(filepath=TSP/CEPS_benchmark/uniform_portgen/00.tsp)


In [3]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=2)
portfolio

Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]

In [5]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    t_c=400,
    calculate_features=False,
    estimator=None,
)
aac.configure()

[2024-12-30 20:48:35] DEBUG     AAC(prefix=config, iter=1, configuration_time=[400.00 400.00])
[2024-12-30 20:48:35] DEBUG     Portfolio.evaluate(config;aac_iter=1)
[2024-12-30 20:48:35] DEBUG     Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]
[2024-12-30 20:48:35] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/01.tsp))
[2024-12-30 20:48:35] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/01.tsp))
[2024-12-30 20:48:35] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/02.tsp))
[2024-12-30 20:48:35] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/cluster/02.tsp))
[2024-12-30 20:48:35] DEBUG     solve(prefix=confi

Portfolio(size=2)[Solver(id=52980316141715150), Solver(id=473831520656465062)]

In [4]:
# db = DB()
db = DB("database/run-plain-50-924800.db")

In [5]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,...,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,822075595471039179,TSP/CEPS_benchmark/cluster_netgen/01.tsp,9080004.0,0.011018,1.411569,1.481377,3.129632,0.897300,3.118615,0.605720,...,0.055556,0.500000,0.5,0.500000,0.944444,0.85250,0.29625,0.32,341.308008,0.67
1,411598327185181776,TSP/CEPS_benchmark/cluster_netgen/02.tsp,11020488.0,0.006047,1.536236,1.561506,3.138415,0.923695,3.132368,0.591541,...,0.083333,0.500000,0.5,0.500000,0.916667,0.86750,0.26750,0.19,390.031034,0.42
2,56462699908304675,TSP/CEPS_benchmark/cluster_netgen/03.tsp,9746976.0,0.005927,1.392946,1.468175,3.140087,0.912639,3.134160,0.621614,...,0.166667,0.500000,0.5,0.500000,0.833333,0.87125,0.25875,0.22,403.471371,0.42
3,1135796868948507057,TSP/CEPS_benchmark/cluster_netgen/04.tsp,8918339.0,0.003895,1.407044,1.487371,3.135897,0.929052,3.132002,0.624627,...,0.250000,0.500000,0.5,0.500000,0.750000,0.90000,0.20250,0.18,398.580695,0.42
4,900769072242699328,TSP/CEPS_benchmark/cluster_netgen/05.tsp,10525131.0,0.004843,1.532593,1.541054,3.139785,0.893347,3.134942,0.579699,...,0.166667,0.500000,0.5,0.500000,0.850000,0.86125,0.28000,0.23,343.126606,0.47
5,2218494142148576373,TSP/CEPS_benchmark/cluster_netgen/06.tsp,11378866.0,0.000030,1.563904,1.532707,3.141496,0.911452,3.141466,0.594668,...,0.055556,0.500000,0.5,0.500000,0.944444,0.85625,0.29125,0.19,409.480210,0.42
6,638039742401470133,TSP/CEPS_benchmark/cluster_netgen/07.tsp,8099809.0,0.000200,1.454258,1.526431,3.141126,0.916145,3.140926,0.600187,...,0.250000,0.500000,0.5,0.500000,0.750000,0.89500,0.21375,0.19,472.771134,0.42
7,1250745597916572136,TSP/CEPS_benchmark/cluster_netgen/08.tsp,9520374.0,0.000503,1.505856,1.523312,3.139432,0.934260,3.138929,0.613309,...,0.166667,0.500000,0.5,0.500000,0.833333,0.86375,0.27500,0.18,441.124394,0.42
8,2855286813102017,TSP/CEPS_benchmark/cluster_netgen/09.tsp,9536693.0,0.000949,1.427310,1.501445,3.139976,0.904546,3.139027,0.602450,...,0.083333,0.500000,0.5,0.500000,0.833333,0.86375,0.28000,0.18,374.586123,0.42
9,1392440215832112593,TSP/CEPS_benchmark/cluster_netgen/10.tsp,10674337.0,0.005924,1.550019,1.546808,3.137950,0.896372,3.132026,0.579498,...,0.166667,0.500000,0.5,0.500000,0.833333,0.91750,0.16500,0.20,409.405108,0.42


In [6]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,473831520656465062,0.25,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.5,...,0.470388,0.888889,1.00,0.6,0.2,0.244898,0.0,1.0,3.0,1.0
2,1724561038518932584,0.30,0.0,1.0,2.0,0.6,0.0,1.0,1.0,0.5,...,0.192348,0.777778,0.50,0.4,0.4,0.571429,0.0,1.0,5.0,0.0
3,388564707472054326,0.65,1.0,0.0,3.0,0.8,0.0,0.0,1.0,1.0,...,0.547346,0.222222,0.50,0.8,0.0,0.377551,1.0,0.0,2.0,0.0
4,2169120517035206132,0.35,1.0,1.0,1.0,0.7,0.0,0.0,0.0,0.0,...,0.197133,0.333333,0.25,1.0,1.0,0.510204,1.0,1.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,1485195866746901139,0.50,1.0,0.0,2.0,0.6,0.0,0.0,1.0,0.5,...,0.600686,0.111111,0.00,0.6,0.6,0.173469,0.0,1.0,3.0,0.0
122,1128357191622349873,0.45,1.0,0.0,1.0,0.5,0.0,1.0,0.0,0.0,...,0.468471,0.666667,0.75,0.6,0.4,0.704082,1.0,1.0,4.0,0.0
123,1990645238621971055,0.75,1.0,1.0,1.0,0.8,0.0,0.0,0.0,0.5,...,0.935587,0.111111,0.50,0.0,0.4,0.020408,0.0,0.0,4.0,1.0
124,1892086461324458362,0.40,1.0,1.0,3.0,0.4,0.0,0.0,0.0,1.0,...,0.329443,0.222222,0.50,0.4,0.4,0.765306,1.0,1.0,0.0,1.0


In [7]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,config;aac_iter=1_52980316141715150_8220755954...,config;aac_iter=1,52980316141715150,822075595471039179,100.00,42.273140,0,0,0
1,config;aac_iter=1_52980316141715150_4115983271...,config;aac_iter=1,52980316141715150,411598327185181776,100.00,16.826909,0,0,0
2,config;aac_iter=1_52980316141715150_5646269990...,config;aac_iter=1,52980316141715150,56462699908304675,100.00,16.275317,0,0,0
3,config;aac_iter=1_473831520656465062_113579686...,config;aac_iter=1,473831520656465062,1135796868948507057,100.00,10.000000,0,0,0
4,config;aac_iter=1_52980316141715150_1135796868...,config;aac_iter=1,52980316141715150,1135796868948507057,100.00,16.191197,0,0,0
...,...,...,...,...,...,...,...,...,...
6495,test9_627876081101248227_1765803814609198146,test9,627876081101248227,1765803814609198146,100.00,10.000000,0,0,0
6496,test9_627876081101248227_1239352391178630764,test9,627876081101248227,1239352391178630764,7.07,7.070000,0,0,0
6497,test9_896004077549916659_396111139638522607,test9,896004077549916659,396111139638522607,100.00,10.000000,0,0,0
6498,test9_896004077549916659_310544774750200091,test9,896004077549916659,310544774750200091,100.00,10.000000,0,0,0


In [19]:
results.loc[results["prefix"].str.startswith("test")].groupby(["instance_id", "prefix"])["cost"].min().reset_index().head(10)

,instance_id,prefix,cost
0,1018017649469757748,test0,2.08
1,1018017649469757748,test1,100.00
2,1018017649469757748,test2,7.70
3,1018017649469757748,test3,3.36
4,1018017649469757748,test4,100.00
5,1018017649469757748,test5,100.00
6,1018017649469757748,test6,100.00
7,1018017649469757748,test7,5.13
8,1018017649469757748,test8,100.00
9,1018017649469757748,test9,7.00


In [21]:
results.loc[results["prefix"].str.startswith("test")].groupby(["instance_id", "prefix"])["cost"].min().reset_index().sort_values(["instance_id", "cost"]).groupby("instance_id")["cost"].nth(4)

instance_id
1018017649469757748    7.70
1239352391178630764    3.87
146957612122462024     3.75
1765803814609198146    3.32
2137644624762416652    1.59
244685964131091250     0.50
310544774750200091     1.88
396111139638522607     0.54
648058674037521772     7.49
730618644385552802     0.58
Name: cost, dtype: float64

In [ ]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)